## Load Data ##

In [1]:
! pip install -qU tensorflow_datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
DATA_DIR = '../input/eurosat/'

ds, ds_info = tfds.load('eurosat/rgb',
                        with_info=True,
                        split='train',
                        data_dir=DATA_DIR)

tfds.show_examples(ds, ds_info);

In [4]:
print(ds_info)

## Construct Pipeline ##

In [5]:
BATCH_SIZE = 16
AUTO = tf.data.experimental.AUTOTUNE
SHUFFLE_BUFFER = int(ds_info.splits['train'].num_examples * 0.7)

ds_train, ds_valid = tfds.load('eurosat/rgb',
                               split=['train[:70%]', 'train[70%:]'],
                               data_dir=DATA_DIR,
                               as_supervised=True)

def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

ds_train = (ds_train
            .map(preprocess, AUTO)
            .cache()
            .shuffle(SHUFFLE_BUFFER)
            .repeat()
            # Augmentations go here .map(augment, AUTO)
            .batch(BATCH_SIZE, drop_remainder=True)
            .prefetch(AUTO))

ds_valid = (ds_valid
            .map(preprocess, AUTO)
            .cache()
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

## Model ##

In [6]:
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import AveragePooling2D, Dropout, Dense, Flatten

In [7]:
model_name = 'VGG16'
!mkdir './VGG16'
save_VGG16 = './VGG16/'

In [9]:
NUM_CLASSES = ds_info.features['label'].num_classes

In [10]:
vgg16_base = VGG16(include_top=False, pooling=None, input_shape=(64, 64, 3))

inner = vgg16_base.output

inner = Flatten()(inner)
inner = Dense(units=256, activation='relu')(inner)
inner = Dropout(0.2)(inner)
inner = Dense(units=NUM_CLASSES, activation='softmax')(inner)

model = Model(inputs=vgg16_base.input, outputs=inner)

In [11]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

In [13]:
EPOCHS = 20
STEPS_PER_EPOCH = int(ds_info.splits['train'].num_examples * 0.7) // BATCH_SIZE

early_stopping = tf.keras.callbacks.EarlyStopping(patience=7, min_delta=0.001, restore_best_weights=True)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[early_stopping],
)

In [14]:
model.save('EuroSAT_VGG16.h5')

In [16]:
import pandas as pd

train_stats = pd.DataFrame(history.history)

train_stats.sort_values(by=['sparse_categorical_accuracy'], inplace=False, ascending=False)

In [17]:
train_stats.plot(y=['sparse_categorical_accuracy', 'val_sparse_categorical_accuracy'], kind='line')

In [18]:
train_stats.plot(y=['loss', 'val_loss'], kind='line')